# How to run wastebot

This notebook is to walk through how to run the wastebot. 

## Import modules
A module is a set of functions that can do lots of useful stuff. Kevin and Jonathan made a lot of convenient functions and packed them as modules, which are 
 - robot module 
 - procedures module
 
but `procedures` module call `robot` module internally, so you only need to import this module (more specifically, `DummyExperiment` function)

In a nutshell, you call this section everytime you run the robot 

The next cell also contains all the python module required to analyse the data, such as:
- Communication with R
- Creation of boxplot
- Creation of 3D plots
- Creation of widgets (selection of parameter and dynamic plots)

In [1]:
import os
import sys
import inspect
import time
import csv
import numpy as np
import datetime
import subprocess 

# OpenCV for image acquistion from webcan and subsequent analysis
import cv2

# load Experiment1 procedures module, which will allow access to all general
# procedure functions and those from Experiment1
#from procedures.procedures import Experiment1
from procedures.procedures import DummyExperiment
# add root path to access project modules
HERE_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(HERE_PATH)

#allowed communication with R
from scipy.integrate import odeint
from matplotlib import pyplot as plt

#import for boxplot
import string
import plotly.plotly as py
import plotly.graph_objs as go

#from mpl_toolkits.mplot3d import Axes3D # for 3d plot
% matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()
sns.set_context("poster")
sns.set_style("whitegrid")

## dynamic plot
from ipywidgets import widgets, Button, Layout
from IPython.display import display

## import packages to communicate with R
from numpy import *
import scipy as sp
from pandas import *
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
#import pandas.rpy.common as com

%load_ext rpy2.ipython
from rpy2.robjects import r, pandas2ri
from rpy2 import robjects
from rpy2.robjects.packages import importr

## import ggplot (python package)
#from ggplot import *

# import R packages

d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}

from rpy2.robjects.packages import importr
base = importr('base')
utils = importr('utils')
gc = importr('growthcurver')
plyr = importr('plyr')
ggplot2 = importr('ggplot2')
reshape2 = importr('reshape2')
dat_tab = importr('data.table')
stats = importr('stats', robject_translations = d)
from rpy2.robjects import pandas2ri
pandas2ri.activate()

## Initialising the robot
Robot needs a configuration file to know what's available or not (e.g. Z-axis is available? A pipette is equipped?, etc).  

In [3]:
# create general procedures object, no need to link configfiles
# this initializes all parts of the robot
exp1 = DummyExperiment(robot_configfile="robot/robot_config.json")

# define initial position for X,Y,Z
init_x = 0
init_y = 0
init_z = 50
# pitch between wells cf. http://ibidi.com/xtproducts/en/ibidi-Labware/m-Plates/m-Plate-24-Well
well_pitch = 19.30 

INFO:commanduino.CommandManager:Found CommandManager on port "COM17", init time was 1.029 seconds
INFO:commanduino.CommandManager:Device "Y" with id "Y" and of type "LINEARACCELSTEPPER" found in 0.016s
INFO:commanduino.CommandManager:Device "Y" with id "Y" and of type "LINEARACCELSTEPPER" found in the register, creating it
INFO:commanduino.CommandManager:Device "X" with id "X" and of type "LINEARACCELSTEPPER" found in 0.016s
INFO:commanduino.CommandManager:Device "X" with id "X" and of type "LINEARACCELSTEPPER" found in the register, creating it
INFO:commanduino.CommandManager:Device "S" with id "S" and of type "LINEARACCELSTEPPER" found in 0.015s
INFO:commanduino.CommandManager:Device "S" with id "S" and of type "LINEARACCELSTEPPER" found in the register, creating it
INFO:commanduino.CommandManager:Device "Z" with id "Z" and of type "LINEARACCELSTEPPER" found in 0.015s
INFO:commanduino.CommandManager:Device "Z" with id "Z" and of type "LINEARACCELSTEPPER" found in the register, creati

# Moving robot
There are a number of functions defined in `robot.py`, but what we need is essentially only one: `move_to()` 

For more functions, see `robot.py` in `C:\Users\turbidostat\Desktop\wastebot\wastebot_complete\software\python\robot`

In [7]:
exp1.robot.move_to([150,200,0])

In [4]:
exp1.robot.dispose_tip()

In [6]:
exp1.robot.home()

# Using the Pipette
Functions are from pipette.py and explained in the documentation. The main functions are 'aspirate(volume_in_uL)' and 'dispense(volume_in_uL)'. 

The speed and acceleration / deceleration of the pipette can also be set on the fly using the 'speed(steps_per_sec)' and 'acceleration(steps_per_sec_sq)' functions.

In [ ]:
#aspirating 1000uL
exp1.robot.p.aspirate(1000)

In [17]:
#dispensing 300uL
exp1.robot.p.dispense(300)

'Volume 300uL dispensed from pipette'

In [12]:
#dispensing 100uL
exp1.robot.p.dispense(100)

'Volume 100uL dispensed from pipette'

In [18]:
#we can change the speed. The default is 6000.
exp1.robot.p.speed(6000)

In [16]:
#we can change the acceleration. The default is 2000.
exp1.robot.p.acceleration(1000)

In [ ]:
#you can use this home function if you want to return the pipette to zero using the endstop.
#it's faster to use the dispense function with the remaining volume though
exp1.robot.p.home()

# Using the Shaker
Functions are from shaker.py and explained in the documentation. The main function is 'shake(shake_angle, n_shakes, interval_angle, n_sets, shake_speed, interval_speed, acceleration)'. There is also a function for a single rotation, 'rotate(angle_in_degrees)' and a function for moving to the closest home position 'zero()'.

The speed and acceleration / deceleration of the shaker can also be set on the fly using the 'speed(steps_per_sec)' and 'acceleration(steps_per_sec_sq)' functions. This doesn't affect the shake function, and you must set the speed and acceleration after using the shake function.

In [6]:
#Let's do a complete shake operation
#Rapid shaking 10x back and forth by 10 degrees. 
#Using 6 sets of shaking, 60 degrees apart.
#shaking speed = 6000, interval speed = 250 and acceleration = 12000.
exp1.robot.s.shake(10, 10, 60, 6, 6000, 250, 12000)
#the shake operation ends with the zero() function. Otherwise you should call it once finished using the shaker.

In [5]:
#we can do a single rotation. 
#Use the speed and acceleration functions to set the speed and acceleration
exp1.robot.s.rotate(30)

In [9]:
#we can set the acceleration.
#ignored and reset by the shake function
exp1.robot.s.acceleration(1000)

In [8]:
#we can set the speed.
#ignored and reset by the shake function
exp1.robot.s.speed(1000)

In [19]:
#we can rotate to the closest home position without using the limit switch
exp1.robot.s.zero()